In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.transforms import Affine2D
import matplotlib.transforms as transforms
import matplotlib
import pandas as pd
import os
import os.path
from scipy import stats
import rpy2
import pingouin as pg

In [2]:
dname = os.getcwd()
dname

'c:\\Users\\Jirko\\Desktop\\Hip_Lab\\analysis_scripts\\part_level_analysis'

In [3]:
def load_parameters(function, levels_list, n_params):

     all_popt = np.full((49,n_params, len(levels_list)), np.nan)

     # loop over each level in the hierarchy
     for j, level in enumerate(levels_list):
          print(level)
          # load the model params for each participant
          df = pd.read_csv(dname + "/params_trialwise/" + level + "/" + function + ".csv")
          # convert to a numpy array
          all_popt[:,:,j] = df.to_numpy()[:,1:]

     return all_popt

load data for the sigmoid function

In [4]:
levels_list = ["first", "second", "third"]
all_popt_sig = load_parameters("sigmoid", levels_list, 2)
np.set_printoptions(suppress=True)
all_popt_sig[:,1,:]

first
second
third


array([[ 0.16313757,  0.01419942,  0.00428915],
       [ 0.0589399 ,  0.05259373,  0.06015026],
       [ 0.31622844,  0.01335061,  8.19469091],
       [ 0.36777479,  0.0273126 ,  0.01651874],
       [ 0.07297063,  0.00806406,  0.01239857],
       [ 0.56075114,  0.01709113,  0.00800739],
       [22.41925197, 18.95729278, -0.00175428],
       [ 0.05525069,  0.04876343,  0.06339958],
       [ 0.84881293,  0.00426701,  0.00356616],
       [ 0.05319436,  0.05241972,  0.1168471 ],
       [ 0.00122941, -0.0050139 ,  0.00891045],
       [19.96004609,  9.35180579,  0.06137927],
       [ 0.02771156,  0.00981549,  0.01278051],
       [20.29441186,  0.00278962, -0.00364497],
       [20.14691063,  0.01346834,  0.00668531],
       [ 0.20133064,  0.20361201,  0.08450377],
       [ 0.08910803,  0.04672859,  0.72841137],
       [ 0.1117511 , 20.07174176,  8.87025709],
       [17.29777779,  0.00830623, -0.00028974],
       [ 9.8239217 , -0.0061689 , -0.0167701 ],
       [ 9.88007994,  0.30766782,  0.194

load the data for the step-function

In [41]:
levels_list = ["first", "second", "third"]
all_popt_step = load_parameters("linear_step_2", levels_list, 2)
np.set_printoptions(suppress=True)
all_popt_step[:,1,:]

first
second
third


array([[ 15.8974359 ,  81.20657067, 128.        ],
       [ 39.42418426,  60.10586313,  63.45278806],
       [  8.15999999,  88.0503049 , 128.        ],
       [  7.77777776,  80.72423399, 128.        ],
       [ 41.35590204, 119.67599412, 128.        ],
       [ 11.76744124,  25.22831051, 128.        ],
       [  6.76862287, 116.44851964, 128.        ],
       [ 59.85507227,  76.2524272 ,  74.13932219],
       [  4.99999986, 128.        , 128.        ],
       [ 30.40192929,  49.66216158,  52.59036169],
       [128.        , 128.        , 128.        ],
       [  6.49999989,  32.40793114,  39.96108935],
       [ 74.5806277 , 128.        , 128.        ],
       [ 33.05804683, 128.        , 128.        ],
       [ 16.08602147, 118.39039492, 128.        ],
       [ 13.20967742,  12.74509805,  23.12299466],
       [ 22.06395351,  55.59024353,   8.0000001 ],
       [ 11.24444442,  20.49999999,  21.08917195],
       [ 96.25192793, 128.        , 128.        ],
       [ 16.80898878, 128.     

In [6]:
# calculate the differences between the two parameters
differences = np.array([all_popt_step[:,0,i] - all_popt_step[:,1,i] for i in range(all_popt_step.shape[2])])
differences = differences.T
distance = np.absolute(differences)
distance.shape

(49, 3)

In [26]:
df_distances = pd.DataFrame(distance, columns=["first", "second", "third"])
df_distances.insert(0, 'ID', range(len(df_distances)))
df_distances.head()
for column in df_distances.iloc[:,1:]:
    print(column)
    print(df_distances[column].median())
    k2, p = stats.shapiro(df_distances[column])
    print(k2, p)

first
15.897435895437674
0.733368992805481 4.327932145997693e-08
second
69.19913416866069
0.875976026058197 9.861315629677847e-05
third
93.6607081803035
0.799263060092926 1.026049744723423e-06


make into long format

In [13]:
df_dist_long = pd.melt(df_distances,
                        id_vars = "ID", 
                        value_vars=["first", "second", "third"], 
                        value_name="Distance", 
                        var_name="Level")
df_dist_long.head()

,ID,Level,Distance
0,0,first,15.897436
1,1,first,39.424184
2,2,first,8.160000
3,3,first,7.777778
4,4,first,41.355902


Run the anova on the distances

In [19]:
res = pg.friedman(data = df_dist_long, dv="Distance", within="Level", subject="ID")
res

,Source,W,ddof1,Q,p-unc
Friedman,Level,0.520899,2,51.048128,8.223180e-12


In [24]:
post_hoc = pg.pairwise_tests(data = df_dist_long, 
                            dv="Distance",
                            within="Level",
                            subject="ID", 
                            parametric=False,
                            padjust="bonf",
                            return_desc=True)
post_hoc

,Contrast,A,B,mean(A),std(A),mean(B),std(B),Paired,Parametric,W-val,alternative,p-unc,p-corr,p-adjust,hedges
0,Level,first,second,26.329644,30.096700,72.403763,44.628323,True,False,89.0,two-sided,1.966941e-07,5.900822e-07,bonf,-1.201006
1,Level,first,third,26.329644,30.096700,81.991582,47.654671,True,False,69.0,two-sided,6.612981e-08,1.983894e-07,bonf,-1.385685
2,Level,second,third,72.403763,44.628323,81.991582,47.654671,True,False,231.0,two-sided,1.642773e-02,4.928318e-02,bonf,-0.206053


In [42]:
df_b_2 = pd.DataFrame(all_popt_step[:,1,:], columns=["first", "second", "third"])
df_b_2.insert(0, 'ID', range(len(df_b_2)))
df_b_2.head()
for column in df_b_2.iloc[:,1:]:
    print(column)
    print(df_b_2[column].median())
    k2, p = stats.shapiro(df_b_2[column])
    print(k2, p)

first
15.897435895437674
0.7283564209938049 3.472906229262662e-08
second
80.724233992896
0.8491699695587158 1.741409687383566e-05
third
128.0
0.709193766117096 1.5317544566073593e-08


In [44]:
df_b_2_long = pd.melt(df_b_2,
                        id_vars = "ID", 
                        value_vars=["first", "second", "third"], 
                        value_name="value", 
                        var_name="Level")
df_b_2_long.head(15)

,ID,Level,value
0,0,first,15.897436
1,1,first,39.424184
2,2,first,8.160000
3,3,first,7.777778
4,4,first,41.355902
5,5,first,11.767441
6,6,first,6.768623
7,7,first,59.855072
8,8,first,5.000000
9,9,first,30.401929


In [45]:
res = pg.friedman(data = df_b_2_long, dv="value", within="Level", subject="ID")
res

,Source,W,ddof1,Q,p-unc
Friedman,Level,0.684247,2,67.05618,2.747487e-15


In [47]:
post_hoc = pg.pairwise_tests(data = df_b_2_long, 
                            dv="value",
                            within="Level",
                            subject="ID", 
                            parametric=False,
                            padjust="bonf",
                            return_desc=True)
post_hoc

,Contrast,A,B,mean(A),std(A),mean(B),std(B),Paired,Parametric,W-val,alternative,p-unc,p-corr,p-adjust,hedges
0,Level,first,second,26.597350,29.923271,77.603919,45.792120,True,False,25.0,two-sided,7.959835e-09,2.387951e-08,bonf,-1.308344
1,Level,first,third,26.597350,29.923271,93.292092,46.503374,True,False,12.0,two-sided,3.577266e-09,1.073180e-08,bonf,-1.692291
2,Level,second,third,77.603919,45.792120,93.292092,46.503374,True,False,65.0,two-sided,7.296897e-05,2.189069e-04,bonf,-0.337282
